# StockCharts

This notebook is meant to collect data from stockcharts.com from Industry and Sector performances.  

To Do:  
* Build Sector and Industry Dictionaries  
* Build Beautiful Soup data pull  
* Build dataframe  
* Load to PostgreSQL using SQLAlchemy  
* Calculate dollar volumes by cross-referencing Finviz or yfinance Shares Oustanding data

In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup as soup

# Instantiate an Options object and add the --headless argument
# This runs the browser in the background
opts = Options()
opts.add_argument(" --headless")

# Set the location of the webdriver
chrome_driver='C:\\Users\\srobi\\Projects\\chromedriver.exe'

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts,executable_path=chrome_driver)

# To scrape a url rather than a local file use this
my_url = 'https://stockcharts.com/freecharts/industrysummary.html#&O=6'
driver.get(my_url)

# Put the page source into a variable and create a BS object from it, parse HTML
page_html = driver.page_source
page_soup = soup(page_html, "html.parser")

# Close the browser
driver.quit()

In [2]:
# Tests to see if html has been parsed properly
print('\nTesting html parsing...')
print(page_soup.title.get_text())
print(page_soup.h1.text) # shows header information
print(page_soup.p.text) # shows paragraph information
print(page_soup.body.span.text)
print('Testing completed.\n')


Testing html parsing...
Industry Summary | StockCharts.com
Industry Summary
Simple Moving Averages: 20-period / 50-period / 200-period
Welcome
Testing completed.



In [3]:
# Grabs each product
index_symbols = page_soup.findAll("span",{"class":"index-symbol"})
print(f'There are {len(index_symbols)} total.') # find total number of containers
index_symbol = index_symbols[0].text # look at the first container
print(f'Here is the first title: \n{index_symbol}\n')

# paragraphs = page_soup.findAll("div",{"class":"article_teaser_body"})
# print(f'There are {len(paragraphs)} paragraphs total.') # find total number of containers
# news_p = paragraphs[0].text # look at the first container
# print(f'Here is the first paragraph: \n{news_p}\n')

There are 104 total.
Here is the first title: 
$DJUSAV



In [4]:
# Grabs each product
index_names = page_soup.findAll("span",{"class":"index-name"})
print(f'There are {len(index_names)} titles total.') # find total number of containers
index_name = index_names[0].text # look at the first container
print(f'Here is the first title: \n{index_name}\n')

There are 104 titles total.
Here is the first title: 
Media Agencies



In [5]:
index_symbols_list = []
index_names_list = []

for x in range(len(index_names)):
    index_symbols_list.append(index_symbols[x].text)
    index_names_list.append(index_names[x].text)

index_dict = {"index_symbol":index_symbols_list,"index_name":index_names_list}

In [8]:
# Grabs each sector name
sector_names = page_soup.findAll("span",{"class":"panel-title"})
print(f'There are {len(sector_names)} sectors total.') # find total number of containers
sector_name = sector_names[0].text # look at the first container
print(f'Here is the first sector: \n{sector_name}\n')

There are 11 titles total.
Here is the first title: 
Communication Services



In [54]:
# Organizes all sectors, industries, and industry indices together

sectors = page_soup.findAll('div',{"id":"majorMarkets-panel"})

industry_list = []

for sector in sectors:
#     print("\n")
#     print(sector.div.span.text)
    sector0 = sector.div.span.text
    industry_indices = sector.findAll('span',{"class":"index-symbol"})
    industry_names = sector.findAll('span',{"class":"index-name"})
    for x in range(len(industry_indices)):
#         print(f"{industry_indices[x].text} - {industry_names[x].text}")
        industry_list.append({"sector":sector0,"index":industry_indices[x].text,"industry_name":industry_names[x].text})



Communication Services
$DJUSAV - Media Agencies
$DJUSBC - Broadcasting & Entertainment
$DJUSFC - Fixed Line Telecommunications
$DJUSNS - Internet
$DJUSPB - Publishing
$DJUSWC - Mobile Telecommunications


Consumer Discretionary
$DJUSAT - Auto Parts
$DJUSAU - Automobiles
$DJUSBE - Business Training & Employment Agencies
$DJUSCA - Gambling
$DJUSCF - Clothing & Accessories
$DJUSCS - Specialized Consumer Services
$DJUSFH - Furnishings
$DJUSFT - Footwear
$DJUSHB - Home Construction
$DJUSHD - Durable Household Products
$DJUSHI - Home Improvement Retailers
$DJUSLG - Hotels
$DJUSRA - Apparel Retailers
$DJUSRB - Broadline Retailers
$DJUSRP - Recreational Products
$DJUSRQ - Recreational Services
$DJUSRS - Specialty Retailers
$DJUSRU - Restaurants & Bars
$DJUSTT - Travel & Tourism
$DJUSTY - Toys
$DWCTIR - Tires


Consumer Staples
$DJUSCM - Personal Products
$DJUSDB - Brewers
$DJUSFD - Food Retailers & Wholesalers
$DJUSFP - Food Products
$DJUSHN - Nondurable Household Products
$DJUSRD - Drug Ret

In [55]:
df2 = pd.DataFrame(industry_list)
df2

,sector,index,industry_name
0,Communication Services,$DJUSAV,Media Agencies
1,Communication Services,$DJUSBC,Broadcasting & Entertainment
2,Communication Services,$DJUSFC,Fixed Line Telecommunications
3,Communication Services,$DJUSNS,Internet
4,Communication Services,$DJUSPB,Publishing
...,...,...,...
99,Technology,$DWCREE,Renewable Energy Equipment
100,Utilities,$DJUSGU,Gas Distribution
101,Utilities,$DJUSMU,Multiutilities
102,Utilities,$DJUSVE,Conventional Electricity


In [53]:
# print(page_soup.find('div',{"id":"majorMarkets-panel"}).prettify())